In [1]:
from google.colab import drive
drive.mount('/gdrive')
!ls /gdrive

# Set the right directory
# Must change to right directory to run!
%cd /gdrive/My\ Drive/Colab\ Notebooks/RL/city_learn_learn2learn

Mounted at /gdrive
MyDrive
/gdrive/My Drive/Colab Notebooks/RL/city_learn_learn2learn


In [2]:
! pip3 install cherry-rl
# ! pip3 install cherry.algorithms
# ! pip3 install cherry.models.robotics
! pip3 install tqdm
! pip3 install learn2learn

     |████████████████████████████████| 40kB 4.6MB/s 
  Created wheel for cherry-rl: filename=cherry_rl-0.1.4-cp37-none-any.whl size=57843 sha256=e5f9cec51f6e04d5169edd809324cd0c49927608dc76205001b4c7e512cf5da5
  Stored in directory: /root/.cache/pip/wheels/5f/cc/9e/54f8e0c8adbae7287df28415183c2c5539a33811c0a270b309
Successfully built cherry-rl
     |████████████████████████████████| 1.4MB 7.5MB/s 
     |████████████████████████████████| 2.9MB 27.3MB/s 
     |████████████████████████████████| 174kB 1.9MB/s 
     |████████████████████████████████| 102kB 12.2MB/s 
     |████████████████████████████████| 61kB 9.0MB/s 
     |████████████████████████████████| 61kB 9.2MB/s 
     |████████████████████████████████| 1.4MB 58.0MB/s 
     |████████████████████████████████| 112kB 55.0MB/s 
     |████████████████████████████████| 3.2MB 46.7MB/s 
  Created wheel for learn2learn: filename=learn2learn-0.1.5-cp37-cp37m-linux_x86_64.whl size=918065 sha256=23dde866669b951960a6b14c221b502d63c967dc14f75729

In [18]:
import cherry as ch
from cherry._utils import _min_size, _istensorable
from cherry.envs.base import Wrapper
from cherry.envs.utils import is_vectorized

from collections.abc import Iterable


def flatten_episodes(replay, episodes, num_workers):
    """
    TODO: This implementation is not efficient.
    NOTE: Additional info (other than a transition's default fields) is simply copied.
    To know from which worker the data was gathered, you can access sars.runner_id
    TODO: This is not great. What is the best behaviour with infos here ?
    """
    flat_replay = ch.ExperienceReplay()
    worker_replays = [ch.ExperienceReplay() for w in range(num_workers)]
    flat_episodes = 0
    for sars in replay:
        state = sars.state.view(_min_size(sars.state))
        action = sars.action.view(_min_size(sars.action))
        reward = sars.reward.view(_min_size(sars.reward))
        next_state = sars.next_state.view(_min_size(sars.next_state))
        done = sars.done.view(_min_size(sars.done))
        fields = set(sars._Transition__fields) - {'state', 'action', 'reward', 'next_state', 'done'}
        infos = {f: getattr(sars, f) for f in fields}
        for worker in range(num_workers):
            infos['runner_id'] = worker
            # The following attemps to split additional infos. (WIP. Remove ?)
            # infos = {}
            # for f in fields:
            #     value = getattr(sars, f)
            #     if isinstance(value, Iterable) and len(value) == num_workers:
            #         value = value[worker]
            #     elif _istensorable(value):
            #         tvalue = ch.totensor(value)
            #         tvalue = tvalue.view(_min_size(tvalue))
            #         if tvalue.size(0) == num_workers:
            #             value = tvalue[worker]
            #     infos[f] = value
            worker_replays[worker].append(state[worker],
                                          action[worker],
                                          reward[worker],
                                          next_state[worker],
                                          done[worker],
                                          **infos,
                                          )
            if bool(done[worker]):
                flat_replay += worker_replays[worker]
                worker_replays[worker] = ch.ExperienceReplay()
                flat_episodes += 1
            if flat_episodes >= episodes:
                break
        if flat_episodes >= episodes:
            break
    return flat_replay


class Runner(Wrapper):

    """
    Runner wrapper.
    TODO: When is_vectorized and using episodes=n, use the parallel
    environmnents to sample n episodes, and stack them inside a flat replay.
    """

    def __init__(self, env):
        super(Runner, self).__init__(env)
        self.env = env
        self._needs_reset = True
        self._current_state = None

    def reset(self, *args, **kwargs):
        self._current_state = self.env.reset(*args, **kwargs)
        self._needs_reset = False
        return self._current_state

    def step(self, action, *args, **kwargs):
        # TODO: Implement it to be compatible with .run()
        raise NotImplementedError('Runner does not currently support step.')

    def run(self,
            get_action,
            steps=None,
            episodes=None,
            render=False):
        """
        Runner wrapper's run method.
        """
        ####
        check = True
        ####
        if steps is None:
            steps = float('inf')
            if self.is_vectorized:
                self._needs_reset = True
        elif episodes is None:
            episodes = float('inf')
        else:
            msg = 'Either steps or episodes should be set.'
            raise Exception(msg)

        replay = ch.ExperienceReplay()
        collected_episodes = 0
        collected_steps = 0
        while True:
            if collected_steps >= steps or collected_episodes >= episodes:
                if self.is_vectorized and collected_episodes >= episodes:
                    replay = flatten_episodes(replay, episodes, self.num_envs)
                    self._needs_reset = True
                return replay
            if self._needs_reset:
                self.reset()
            info = {}
            #
            if check:
              print (self._current_state.shape)
              check = False
            #
            # print (self.current_action)
            try:
              get_action(self._current_state)
            except:
              print (self.current_action)
            action = get_action(self._current_state)
            # print (type(get_action))
            if isinstance(action, tuple):
                skip_unpack = False
                if self.is_vectorized:
                    if len(action) > 2:
                        skip_unpack = True
                    elif len(action) == 2 and \
                            self.env.num_envs == 2 and \
                            not isinstance(action[1], dict):
                        # action[1] is not info but an action
                        action = (action, )
                if not skip_unpack:
                    if len(action) == 2:
                        info = action[1]
                        action = action[0]
                    elif len(action) == 1:
                        action = action[0]
                    else:
                        msg = 'get_action should return 1 or 2 values.'
                        raise NotImplementedError(msg)
            old_state = self._current_state
            state, reward, done, _ = self.env.step(action)
            if not self.is_vectorized and done:
                collected_episodes += 1
                self._needs_reset = True
            elif self.is_vectorized:
                collected_episodes += sum(done)
            # replay.append(old_state, action, reward, state, done, **info)
            replay.append(old_state, action, reward, state, done)
            self._current_state = state
            if render:
                self.env.render()
            collected_steps += 1

In [19]:
import math

import cherry as ch
import torch
from torch import nn
from torch.distributions import Normal, Categorical

EPSILON = 1e-6


def linear_init(module):
    if isinstance(module, nn.Linear):
        nn.init.xavier_uniform_(module.weight)
        module.bias.data.zero_()
    return module
    
class DiagNormalPolicy(nn.Module):

    def __init__(self, input_size, output_size, hiddens=None, activation='relu', device='cpu'):
        super(DiagNormalPolicy, self).__init__()
        self.device = device
        if hiddens is None:
            hiddens = [100, 100]
        if activation == 'relu':
            activation = nn.ReLU
        elif activation == 'tanh':
            activation = nn.Tanh
        layers = [linear_init(nn.Linear(input_size, hiddens[0])), activation()]
        for i, o in zip(hiddens[:-1], hiddens[1:]):
            layers.append(linear_init(nn.Linear(i, o)))
            layers.append(activation())
        layers.append(linear_init(nn.Linear(hiddens[-1], output_size)))
        self.mean = nn.Sequential(*layers)
        self.sigma = nn.Parameter(torch.Tensor(output_size))
        self.sigma.data.fill_(math.log(1))

    def density(self, state):
        state = state.to(self.device, non_blocking=True)
        loc = self.mean(state)
        scale = torch.exp(torch.clamp(self.sigma, min=math.log(EPSILON)))
        try:
          Normal(loc=loc, scale=scale)
        except:
          print(loc)
        return Normal(loc=loc, scale=scale)

    def log_prob(self, state, action):
        density = self.density(state)
        return density.log_prob(action).mean(dim=1, keepdim=True)

    def forward(self, state):
        density = self.density(state)
        action = density.sample()
        return action


In [20]:
#!/usr/bin/env python3

"""
Trains a 2-layer MLP with MAML-VPG augmented with the DiCE objective.

Usage:

python examples/rl/maml_dice.py
"""

import random
from pathlib import Path
import cherry as ch
import gym
import numpy as np
import torch
from cherry.algorithms import a2c
from cherry.models.robotics import LinearValue
from torch import optim
from tqdm import tqdm
from gym import spaces

import learn2learn as l2l
import policy_meta
# from policies import DiagNormalPolicy

import citylearn


def weighted_cumsum(values, weights):
    for i in range(values.size(0)):
        values[i] += values[i - 1] * weights[i]
    return values


def compute_advantages(baseline, tau, gamma, rewards, dones, states, next_states):
    # Update baseline
    returns = ch.td.discount(gamma, rewards, dones)
    baseline.fit(states, returns)
    values = baseline(states)
    next_values = baseline(next_states)
    bootstraps = values * (1.0 - dones) + next_values * dones
    next_value = torch.zeros(1, device=values.device)
    return ch.pg.generalized_advantage(tau=tau,
                                       gamma=gamma,
                                       rewards=rewards,
                                       dones=dones,
                                       values=bootstraps,
                                       next_value=next_value)


def maml_a2c_loss(train_episodes, learner, baseline, gamma, tau):
    # Update policy and baseline
    states = train_episodes.state()
    actions = train_episodes.action()
    rewards = train_episodes.reward()
    dones = train_episodes.done()
    next_states = train_episodes.next_state()
    log_probs = learner.log_prob(states, actions)
    weights = torch.ones_like(dones)
    weights[1:].add_(-1.0, dones[:-1])
    weights /= dones.sum()
    cum_log_probs = weighted_cumsum(log_probs, weights)
    advantages = compute_advantages(baseline, tau, gamma, rewards,
                                    dones, states, next_states)
    return a2c.policy_loss(l2l.magic_box(cum_log_probs), advantages)


def main(
        experiment='dev',
        adapt_lr=0.1,
        meta_lr=0.001,
        adapt_steps=1,
        num_iterations=20,
        meta_bsz=20,
        adapt_bsz=20,
        tau=1.00,
        gamma=0.99,
        num_workers=8,
        seed=42,
):
    def make_env():
        '''
        set environment
        '''
        # Contain the lower and upper bounds of the states and actions, to be provided to the agent to normalize the variables between 0 and 1.
        # Can be obtained using observations_spaces[i].low or .high
        # TODO: need to change __init__.py to change some settings

        return gym.make("citylearn:citylearn-v0")
    def get_dim(env):
        '''
        get state and action dimension of the environment
        '''
        building_id = ["Building_1"]
        observations_space, actions_space = env.get_state_action_spaces()
        # observations_spaces, actions_spaces = env.get_state_action_spaces()
        # TODO: need to change when we do multiple buildings
        action_spaces = {"0": a_space for uid, a_space in zip(building_id, actions_space)}
        action_dim = action_spaces["0"].shape[0]
        observation_spaces = {"0":o_space for uid, o_space in zip(building_id, observations_space)}
        state_dim = observation_spaces["0"].shape[0]
        return (action_spaces, action_dim, state_dim)


    env = make_env()
    action_spaces, action_dim, state_dim = get_dim(env)
    print(state_dim, action_dim)
    env = l2l.gym.AsyncVectorEnv([make_env for _ in range(num_workers)])
    # env.seed(seed)
    env = ch.envs.Torch(env)
    
    #TODO: PolicyNetwork(state_dim, action_dim, self.action_spaces[uid], \
    # self.action_scaling_coef, hidden_dim).to(self.device)
    #TODO: I need to incorporate a code where it we amek policy by its states and actionspace
    # for each building.
    policy = DiagNormalPolicy(state_dim, action_dim)
    # policy = policy_meta.PolicyNetwork(state_dim, action_dim, action_spaces["0"])
    meta_learner = l2l.algorithms.MAML(policy, lr=adapt_lr)
    baseline = LinearValue(env.state_size, env.action_size)
    opt = optim.Adam(policy.parameters(), lr=meta_lr)
    all_rewards = []

    for iteration in range(num_iterations):
        iteration_loss = 0.0
        iteration_reward = 0.0
        for task_config in tqdm(env.sample_tasks(meta_bsz), leave=False, desc='Data'):  # Samples a new config
            learner = meta_learner.clone()
            print(task_config)
            env.set_task(task_config)
            env.reset()
            task = Runner(env)

            # Fast Adapt
            for step in range(adapt_steps):
                train_episodes = task.run(learner, episodes=adapt_bsz)
                loss = maml_a2c_loss(train_episodes, learner, baseline, gamma, tau)
                print("this is inner loop loss")
                print(loss)
                learner.adapt(loss)

            # Compute Validation Loss
            valid_episodes = task.run(learner, episodes=adapt_bsz)
            loss = maml_a2c_loss(valid_episodes, learner, baseline, gamma, tau)
            print("this is validation loss")
            print(loss)
            print("")
            iteration_loss += loss
            iteration_reward += valid_episodes.reward().sum().item() / adapt_bsz

        # Print statistics
        print('\nIteration', iteration)
        adaptation_reward = iteration_reward / meta_bsz
        print('adaptation_reward', adaptation_reward)
        all_rewards.append(adaptation_reward)
        print(iteration_loss)
        print(meta_bsz)
        adaptation_loss = iteration_loss / meta_bsz
        print('adaptation_loss', adaptation_loss.item())

        opt.zero_grad()
        adaptation_loss.backward()
        opt.step()


if __name__ == '__main__':
    main()


26 2





Data:   0%|          | 0/20 [00:00<?, ?it/s]

{'zone': 3}
torch.Size([8, 1, 26])
this is inner loop loss
tensor(-217.4843, grad_fn=<NegBackward>)
torch.Size([8, 1, 26])





Data:   5%|▌         | 1/20 [00:06<02:02,  6.46s/it]

this is validation loss
tensor(-210.9073, grad_fn=<NegBackward>)

{'zone': 3}
torch.Size([8, 1, 26])
this is inner loop loss
tensor(-165.8959, grad_fn=<NegBackward>)
torch.Size([8, 1, 26])


...

this is validation loss
tensor(-205.0901, grad_fn=<NegBackward>)


Iteration 6
adaptation_reward -76198.530859375
tensor(nan, grad_fn=<AddBackward0>)
20
adaptation_loss nan





Data:   0%|          | 0/20 [00:00<?, ?it/s]

{'zone': 3}


torch.Size([8, 1, 26])
tensor([[[nan, nan]],

        [[nan, nan]],

        [[nan, nan]],

        [[nan, nan]],

        [[nan, nan]],

        [[nan, nan]],

        [[nan, nan]],

        [[nan, nan]]], grad_fn=<AddBackward0>)


AttributeError: ignored

In [ ]:
# ! python3 maml_l2l.py